# Lesson 8: Cross-validation

So far, we've learned about splitting our data into training and testing sets to validate our models. This helps ensure that the model we create on one sample performs well on another sample we want to predict. 

However, we don't have to use just TWO samples to train and test our models. Instead, we can split our data up into MULTIPLE samples to train and test on multiple segments of the data. This is called CROSS-VALIDATION. This allows us to ensure that our model predicts outcomes over a wider range of circumstances. 

Let's begin by importing our packages.

In [81]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [82]:
! conda install geopandas -yq
import geopandas as gpd
from shapely.geometry import Point, Polygon

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold



# All requested packages already installed.
# packages in environment at /opt/conda:
#
geopandas                 0.3.0                    py36_0    conda-forge


Today we'll be looking at 311 service requests for rodent inspection and abatement aggregated at the Census block level. The data set is already prepared for you and available in the same folder as this assignment. Census blocks are a good geographic level to analyze rodent infestations because they are drawn along natural and human-made boundaries, like rivers and roads, that rats tend not to cross. 

We will look at the 'activity' variable, which indicates whether inspectors found rat burrows during an inspection (1) or not (0). Here we are looking only at inpsections in 2016. About 43 percent on inspections in 2016 led to inspectors finding and treating rat burrows, as you can see below.

In [83]:
data = pd.read_csv('rat_data_2016.csv')

In [84]:
data.columns

Index(['activity', 'alley_condition', 'bbl_hotel', 'bbl_multifamily_rental',
       'bbl_restaurant', 'bbl_single_family_rental', 'bbl_storage',
       'bbl_two_family_rental', 'communitygarden_area', 'communitygarden_id',
       'dcrapermit_addition', 'dcrapermit_demolition', 'dcrapermit_excavation',
       'dcrapermit_new_building', 'dcrapermit_raze', 'impervious_area',
       'month', 'num_mixed_use', 'num_non_residential', 'num_residential',
       'park', 'pct_mixed_use', 'pct_non_residential', 'pct_residential',
       'pop_density', 'sidewalk_grates', 'ssl_cndtn_Average_comm',
       'ssl_cndtn_Average_res', 'ssl_cndtn_Excellent_comm',
       'ssl_cndtn_Excellent_res', 'ssl_cndtn_Fair_comm', 'ssl_cndtn_Fair_res',
       'ssl_cndtn_Good_comm', 'ssl_cndtn_Good_res', 'ssl_cndtn_Poor_comm',
       'ssl_cndtn_Poor_res', 'ssl_cndtn_VeryGood_comm',
       'ssl_cndtn_VeryGood_res', 'tot_pop', 'well_activity', 'WARD'],
      dtype='object')

In [19]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
activity,2606.0,0.431696,0.495408,0.000000,0.000000,0.000000,1.000000,1.000000
alley_condition,2606.0,11.111282,8.900166,0.000000,4.000000,10.000000,16.000000,79.000000
bbl_hotel,2606.0,0.082118,0.376073,0.000000,0.000000,0.000000,0.000000,8.000000
bbl_multifamily_rental,2606.0,1.388718,2.376244,0.000000,0.000000,0.000000,2.000000,22.000000
bbl_restaurant,2606.0,0.569455,1.518526,0.000000,0.000000,0.000000,0.000000,17.000000
bbl_single_family_rental,2606.0,4.709133,8.375165,0.000000,1.000000,2.000000,5.000000,147.000000
bbl_storage,2606.0,0.002686,0.051768,0.000000,0.000000,0.000000,0.000000,1.000000
bbl_two_family_rental,2606.0,0.743668,1.378860,0.000000,0.000000,0.000000,1.000000,15.000000
communitygarden_area,2606.0,18.727920,326.332382,0.000000,0.000000,0.000000,0.000000,11004.319881
communitygarden_id,2606.0,0.242134,3.234069,0.000000,0.000000,0.000000,0.000000,80.000000


Recall from last week that, when we do predictive analysis, we usually are not interested in the relationship between two different variables as we are when we do traditional hypothesis testing. Instead, we're interested in training a model that generates predictions that best fit our target population. Therefore, when we are doing any kind of validation, including cross-validation, it is important for us to choose the metric by which we will evaluate the performance of our models. 

For this model, we will predict the locations of requests for rodent inspection and abatement in the District of Columbia. When we select a validation metric, it's important for us to think about what we want to optimize. For example, do we want to make sure that our top predictions accurately identify places with rodent infestations, so we don't send our inspectors on a wild goose chase? Then we may to look at the models precision, or what proportion of its positive predictions turn out to be positive. Or do we want to make sure we don't miss any infestations? If so, we may want to look at recall, or the proportion of positive cases that are correctly categorized by the model. If we care a lot about how the model ranks our observations, then we may want to look at the area under the ROC curve, or ROC-AUC, while if we care more about how well the model fits the data, or its "calibration," we may want to look at Brier score or logarithmic loss (log-loss).

In the case of rodent inspections, we most likely want to make sure that we send our inspectors to places where they are most likely to find rats and to avoid sending them on wild goose [rat] chases. Therefore, we will optimize for precision, which we will call from the metrics library in scikit-learn. 

The metrics library in scikit-learn provides a number of different options. You should take some time to look at the different metrics that are available to you and consider which ones are most appropriate for your own research


In [85]:
from sklearn.metrics import precision_score

The next important decision we need to make when cross-validating our models is how we will define our "folds." Folds are the independent subsamples on which we train and test the data. Keep in mind that it is important that our folds are INDEPENDENT, which means we must guarantee that there's no overlap between our training and test set (i.e., no observation is in both the training and test set). Independence can also have other implications for how we slice the data, which we will discuss as we progress through this lesson.

One of the most common approaches to cross-validation is to make random splits in the data. This is often referred to as k-fold cross-validation, in which the only thing we define is the number of folds (k) that want to split our sample into. Here, I'll use the KFold function from scikit-learn's model_selection library. Let's begin by importing the library and then taking a look at how it splits our data.

In [86]:
from sklearn.model_selection import KFold

KFold divides our data into a pre-specified number of (approximately) equally-sized folds so that each observation is in the test set once. When we specify that shuffle=True, KFold first shuffles our data into a random order to ensure that the observations are randomly selected. By selecting a random_state, we can ensure that KFold selects observations the same way each time. 

While there are other functions in the model_selection library that will do much of this work for us, KFold will allow us to look at what's going on in the background of our cross-validation process. Let's begin by just looking at how KFold splits our data. Here we split our data into 10 folds each with 10 percent of the data (.1).

In [87]:
cv = KFold(n_splits=10, shuffle=True, random_state=0)
for train_index, test_index in cv.split(data):
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [   0    1    2 ..., 2603 2604 2605] TEST: [   9   22   27   33   53   70   92  104  109  117  121  135  137  156  182
  192  195  196  215  217  224  227  252  259  271  276  289  314  317  326
  333  351  398  399  418  422  427  436  438  443  452  465  478  480  482
  489  518  547  562  563  567  569  578  581  597  609  616  618  619  674
  682  686  700  704  710  711  720  722  728  743  745  746  748  764  778
  795  817  831  855  868  878  880  899  913  916  921  927  933  961  962
  982  983  988  998 1000 1012 1013 1018 1023 1032 1036 1051 1052 1059 1078
 1079 1096 1100 1101 1106 1108 1109 1147 1187 1192 1213 1263 1264 1285 1287
 1300 1323 1326 1327 1371 1396 1418 1421 1432 1452 1484 1507 1515 1520 1544
 1568 1570 1577 1580 1585 1588 1590 1592 1597 1622 1627 1656 1657 1668 1680
 1681 1686 1689 1708 1710 1719 1729 1732 1735 1757 1761 1762 1763 1765 1780
 1783 1790 1798 1803 1814 1816 1818 1820 1821 1827 1832 1836 1853 1873 1874
 1895 1898 1901 1902 1921 1927 1929 19

You can see that ShuffleSplit has selected a random set of observations from the index of our data set for each fold of our cross-validation. Let's look at the size of our training and test set for each fold.

In [88]:
cv = KFold(n_splits=10, shuffle=True, random_state=0)
for train_index, test_index in cv.split(data):
    print("TRAIN:", len(train_index), "TEST:", len(test_index))

TRAIN: 2345 TEST: 261
TRAIN: 2345 TEST: 261
TRAIN: 2345 TEST: 261
TRAIN: 2345 TEST: 261
TRAIN: 2345 TEST: 261
TRAIN: 2345 TEST: 261
TRAIN: 2346 TEST: 260
TRAIN: 2346 TEST: 260
TRAIN: 2346 TEST: 260
TRAIN: 2346 TEST: 260


Now let's try using KFold to train and test our model on 10 different subsets of our data. Below we set our cross-validator as 'cv'. We then loop through the various splits in our data that cv creates and use it to make our training and test sets. We then use our training set to fit a Logistic Regression model and generate predictions from our test set, which we compare to the actual outcomes we observed.

In [89]:
## Define function
cv = KFold(n_splits=10, shuffle=True, random_state=0)

## Create for-loop
for train_index, test_index in cv.split(data):

    ## Define training and test sets
    X_train = data.loc[train_index].drop(['activity', 'month', 'WARD'], axis=1)
    y_train = data.loc[train_index]['activity']
    X_test = data.loc[test_index].drop(['activity', 'month', 'WARD'], axis=1)
    y_test = data.loc[test_index]['activity']
        
    ## Fit model
    clf = LogisticRegression()
    clf.fit(X_train, y_train)

    ## Generate predictions
    predicted = clf.predict(X_test)
    
    ## Compare to actual outcomes and return precision
    print('Precision: '+str(100 * round(precision_score(y_test, predicted),3)))

Precision: 56.9
Precision: 55.8
Precision: 60.4
Precision: 63.5
Precision: 55.4
Precision: 52.8
Precision: 56.9
Precision: 57.1
Precision: 65.8
Precision: 41.0


We can see that, for the most part, about 50 to 60 percent of the inspections our model predicts will lead our inspectors to rat burrows actually do. This is a modest improvement over our inspectors' current performance in the field. Based on these results, if we used our models to determine which locations our inspectors go to in the field, we'd probably see a 10 to 20 point increase in their likelihood of finding rat burrows.

## Exercise 1

Try running the k-fold cross-validation a few times with the same random state. Then try running it a few times with different random states. How do the results change?

In [90]:
cv = KFold(n_splits=20, shuffle=True, random_state=0)
for train_index, test_index in cv.split(data):
    print("TRAIN:", train_index, "TEST:", test_index)


TRAIN: [   0    1    2 ..., 2603 2604 2605] TEST: [  33   53  117  156  192  195  227  252  271  289  314  317  418  422  436
  438  443  478  518  547  562  567  581  616  618  704  711  722  728  745
  748  764  778  795  868  880  899  913  921  927  982  988  998 1012 1013
 1052 1059 1078 1079 1096 1100 1101 1109 1192 1213 1263 1285 1323 1326 1327
 1396 1418 1432 1452 1507 1570 1577 1580 1592 1622 1627 1680 1681 1686 1708
 1710 1719 1729 1735 1757 1762 1763 1765 1803 1818 1821 1832 1853 1873 1874
 1921 1929 1939 1943 1961 1968 1969 1972 1974 1989 1997 2019 2027 2033 2074
 2087 2090 2134 2138 2156 2186 2191 2225 2274 2289 2312 2316 2317 2343 2358
 2386 2416 2419 2429 2468 2498 2510 2543 2550 2555 2597]
TRAIN: [   0    1    2 ..., 2603 2604 2605] TEST: [   9   22   27   70   92  104  109  121  135  137  182  196  215  217  224
  259  276  326  333  351  398  399  427  452  465  480  482  489  563  569
  578  597  609  619  674  682  686  700  710  720  743  746  817  831  855
  878  

In [91]:
cv = KFold(n_splits=20, shuffle=True, random_state=0)
for train_index, test_index in cv.split(data):
    print("TRAIN:", len(train_index), "TEST:", len(test_index))

TRAIN: 2475 TEST: 131
TRAIN: 2475 TEST: 131
TRAIN: 2475 TEST: 131
TRAIN: 2475 TEST: 131
TRAIN: 2475 TEST: 131
TRAIN: 2475 TEST: 131
TRAIN: 2476 TEST: 130
TRAIN: 2476 TEST: 130
TRAIN: 2476 TEST: 130
TRAIN: 2476 TEST: 130
TRAIN: 2476 TEST: 130
TRAIN: 2476 TEST: 130
TRAIN: 2476 TEST: 130
TRAIN: 2476 TEST: 130
TRAIN: 2476 TEST: 130
TRAIN: 2476 TEST: 130
TRAIN: 2476 TEST: 130
TRAIN: 2476 TEST: 130
TRAIN: 2476 TEST: 130
TRAIN: 2476 TEST: 130


In [38]:
## Define function
cv = KFold(n_splits=10, shuffle=True, random_state=0)

## Create for-loop
for train_index, test_index in cv.split(data):

    ## Define training and test sets
    X_train = data.loc[train_index].drop(['activity', 'month', 'WARD'], axis=1)
    y_train = data.loc[train_index]['activity']
    X_test = data.loc[test_index].drop(['activity', 'month', 'WARD'], axis=1)
    y_test = data.loc[test_index]['activity']
        
    ## Fit model
    clf = LogisticRegression()
    clf.fit(X_train, y_train)

    ## Generate predictions
    predicted = clf.predict(X_test)
    
    ## Compare to actual outcomes and return precision
    print('Precision: '+str(100 * round(precision_score(y_test, predicted),3)))


Precision: 56.9
Precision: 55.8
Precision: 60.4
Precision: 63.5
Precision: 55.4
Precision: 52.8
Precision: 56.9
Precision: 57.1
Precision: 65.8
Precision: 41.0


In [92]:
cv = KFold(n_splits=10, shuffle=True, random_state=100)
for train_index, test_index in cv.split(data):
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [   0    1    2 ..., 2603 2604 2605] TEST: [  10   15   23   29   31   54   63   65   71  115  117  121  130  134  147
  151  168  171  176  182  203  229  231  249  257  258  260  264  269  299
  302  309  316  334  339  342  345  349  353  356  362  376  381  384  386
  411  416  433  435  459  469  472  483  490  496  498  510  515  519  526
  530  532  536  551  560  583  585  589  606  635  642  647  650  654  679
  696  709  730  740  752  786  791  801  805  811  823  843  853  881  886
  888  898  899  914  917  918  919  924  932  947  950  952  957  978  980
  981  983  996 1006 1022 1023 1046 1054 1062 1067 1079 1127 1134 1135 1136
 1148 1158 1165 1166 1181 1185 1187 1197 1216 1224 1225 1243 1249 1254 1262
 1265 1270 1306 1313 1329 1339 1352 1356 1360 1369 1378 1380 1383 1389 1394
 1396 1410 1421 1436 1449 1453 1474 1482 1492 1499 1514 1517 1529 1540 1547
 1555 1568 1590 1591 1603 1656 1674 1692 1697 1704 1727 1737 1739 1741 1743
 1762 1775 1776 1783 1788 1799 1810 18

In [93]:
cv = KFold(n_splits=10, shuffle=True, random_state=100)
for train_index, test_index in cv.split(data):
    print("TRAIN:", len(train_index), "TEST:", len(test_index))

TRAIN: 2345 TEST: 261
TRAIN: 2345 TEST: 261
TRAIN: 2345 TEST: 261
TRAIN: 2345 TEST: 261
TRAIN: 2345 TEST: 261
TRAIN: 2345 TEST: 261
TRAIN: 2346 TEST: 260
TRAIN: 2346 TEST: 260
TRAIN: 2346 TEST: 260
TRAIN: 2346 TEST: 260


It's important to point out here that, because we have TIME SERIES data, the same Census blocks may be appearing in our training AND our test sets. This is a challenge to ensuring that our training and test samples are INDEPENDENT. While Rodent Control does not inspect the same blocks every month, some of the same blocks may be re-inspected from month to month depending on where 311 requests are coming from. 

However, this also affords us an opportunity. More than likely, when we make predictions about which inspections will lead our inspectors to rat burrows, we are interested in predicting FUTURE inspections with observations from PAST inspections. In this case, cross-validating over time can be a very good way of looking at how well our models are performing. 

Cross-validating over time requires more than just splitting by month. Rather, we will use observations from each month as a test set and train our models on all PRIOR months. Which we do below.

In [94]:
## Define function
cv = KFold(n_splits=10, shuffle=True, random_state=100)

## Create for-loop
for train_index, test_index in cv.split(data):

    ## Define training and test sets
    X_train = data.loc[train_index].drop(['activity', 'month', 'WARD'], axis=1)
    y_train = data.loc[train_index]['activity']
    X_test = data.loc[test_index].drop(['activity', 'month', 'WARD'], axis=1)
    y_test = data.loc[test_index]['activity']
        
    ## Fit model
    clf = LogisticRegression()
    clf.fit(X_train, y_train)

    ## Generate predictions
    predicted = clf.predict(X_test)
    
    ## Compare to actual outcomes and return precision
    print('Precision: '+str(100 * round(precision_score(y_test, predicted),3)))


Precision: 61.5
Precision: 59.7
Precision: 51.8
Precision: 55.2
Precision: 68.1
Precision: 56.1
Precision: 64.6
Precision: 47.2
Precision: 43.8
Precision: 50.0


## Cross-validation by Month

Let's begin by seeing what our cross-validation sets look like. Below, we loop through each of the sets to see which months end up in our training and test sets. You can see that as we move from month to month, we have more and more past observations in our training set.

In [95]:
months = np.sort(data.month.unique())

for month in range(2,13):
    test = data[data.month==month]
    train = data[(data.month < month)]

    print('Test Month: '+str(test.month.unique()), 'Training Months: '+str(train.month.unique()))

Test Month: [2] Training Months: [1]
Test Month: [3] Training Months: [1 2]
Test Month: [4] Training Months: [1 2 3]
Test Month: [5] Training Months: [1 2 3 4]
Test Month: [6] Training Months: [1 2 3 4 5]
Test Month: [7] Training Months: [1 2 3 4 5 6]
Test Month: [8] Training Months: [1 2 3 4 5 6 7]
Test Month: [9] Training Months: [1 2 3 4 5 6 7 8]
Test Month: [10] Training Months: [1 2 3 4 5 6 7 8 9]
Test Month: [11] Training Months: [ 1  2  3  4  5  6  7  8  9 10]
Test Month: [12] Training Months: [ 1  2  3  4  5  6  7  8  9 10 11]


In [96]:
months = np.sort(data.month.unique())

for month in range(2,13):

    test = data[data.month==month]
    train = data[(data.month < month)]
    X_test = test.drop(['activity', 'month', 'WARD'], axis=1)
    y_test = test['activity']
    X_train = test.drop(['activity', 'month', 'WARD'], axis=1)
    y_train = test['activity']
        
    clf = LogisticRegression()
    clf.fit(X_train, y_train)
    predicted = clf.predict(X_test)
    print('Precision for Month '+str(month)+': '+str(100*round(precision_score(y_test, predicted),3)))

Precision for Month 2: 79.2
Precision for Month 3: 67.9
Precision for Month 4: 48.9
Precision for Month 5: 63.5
Precision for Month 6: 61.3
Precision for Month 7: 67.9
Precision for Month 8: 67.2
Precision for Month 9: 67.6
Precision for Month 10: 57.3
Precision for Month 11: 68.3
Precision for Month 12: 70.5


Our model seems to be performing even better when we cross-validate over months, possibly because we're structuring the cross-validation such that inspections in some of the same blocks appear consistently over time. 

## Exercise 2

Try re-creating this cross-validation, but with the training set restricted to only the 3 months prior to the test set. Now do the same with the last 1 and 2 months. Do the results change?

In [102]:
months = np.sort(data.month.unique())

for month in range(4,13):
    test = data[data.month==month]
    train = data[(data.month==month-3)|(data.month==month-2)|(data.month==month-1)]


    print('Test Month: '+str(test.month.unique()), 'Training Months: '+str(train.month.unique()))

Test Month: [4] Training Months: [1 2 3]
Test Month: [5] Training Months: [2 3 4]
Test Month: [6] Training Months: [3 4 5]
Test Month: [7] Training Months: [4 5 6]
Test Month: [8] Training Months: [5 6 7]
Test Month: [9] Training Months: [6 7 8]
Test Month: [10] Training Months: [7 8 9]
Test Month: [11] Training Months: [ 8  9 10]
Test Month: [12] Training Months: [ 9 10 11]


We may still be concerned about the independence of our training and test sets. In particular, as I've pointed out, the same Census blocks may appear repeatedly in our data over time. In this case, it may be good to cross-validate geographically to make sure that our model is performing well in different parts of the city. In particular, we know that requests for rodent abatement (and rats themselves) are more common in some parts of the city than in others. In particular, rats are more common in the more densely-populated parts of downtown and less common in less densely-populated places like Wards 3, 7, and 8. Therefore, we may be interested in cross-validating by ward. 

Again, this is as simple as looping through each of the 8 wards, holding out each ward as a test set and training the models on observations from the remaining wards.

## Cross-validate by Ward

In [103]:
data.WARD.value_counts().sort_index()

1    419
2    477
3    105
4    496
5    355
6    458
7    154
8    142
Name: WARD, dtype: int64

In [104]:
for ward in np.sort(data.WARD.unique()):

    test = data[data.WARD == ward]
    train = data[data.WARD != ward]
    X_test = test.drop(['activity', 'month', 'WARD'], axis=1)
    y_test = test['activity']
    X_train = test.drop(['activity', 'month', 'WARD'], axis=1)
    y_train = test['activity']
        
    clf = LogisticRegression(class_weight="balanced")
    clf.fit(X_train, y_train)
    predicted = clf.predict(X_test)
    print('Precision for Ward '+str(ward)+': '+str(100*round(precision_score(y_test, predicted),3)))

Precision for Ward 1: 66.3
Precision for Ward 2: 67.2
Precision for Ward 3: 71.8
Precision for Ward 4: 57.5
Precision for Ward 5: 69.4
Precision for Ward 6: 36.0
Precision for Ward 7: 25.0
Precision for Ward 8: 30.6


Here we see that the model performs very well predicting the outcomes of inspections in wards 1 through 4, but less well in wards 5 though 8. In wards 7 and 8 in particular, the model fails to predict any positive cases. This means that our model may be overfit to observations in Wards 1 through 6, and we may want to re-evaluate our approach. 

## Exercise 3

Explore the data and our model and try to come up with some reasons that the model is performing poorly on Wards 7 and 8. Is there a way we can fix the model to perform better on those wards? How might we fix the model?

In [105]:
#because Ward 7 and 8 have more data than other wards. 

## Exercise 4

Now try running some cross-validations with the data from your project. What are some different ways you might slice the data you're using for your project? Try them out here. This will be a good way to begin making progress toward your final submission. 

PLEASE REMEMBER TO SUBMIT THIS HOMEWORK BY CLASS TIME ON THURSDAY.

In [ ]:
#Since we are still merging our data, we will do this once we have it.